In [4]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Global session (shared across threads)
_session = None


def get_session():
    global _session

    if _session is None:
        session = requests.Session()

        retries = Retry(
            total=3,
            backoff_factor=0.5,
            status_forcelist=[429, 500, 502, 503, 504],
            allowed_methods=["GET"],
        )

        adapter = HTTPAdapter(
            max_retries=retries,
            pool_connections=50,
            pool_maxsize=50,
        )

        session.mount("http://", adapter)
        session.mount("https://", adapter)

        _session = session

    return _session

In [5]:
def geocode(address: str, base_url="http://localhost:8080/search") -> dict:
    if not address.strip():
        return {"latitude": None, "longitude": None, "display_name": None}

    # Only look for addresses in Worcester, MA
    min_lon, min_lat, max_lon, max_lat = -71.884043, 42.210053, -71.731237, 42.341187

    params = {
        "q": address.strip(),
        "format": "json",
        "limit": 1,
        # Nominatim bounding box (lon1, lat1, lon2, lat2)
        "viewbox": f"{min_lon},{max_lat},{max_lon},{min_lat}",  # note: top-left, bottom-right
        "bounded": 1,  # restrict results strictly to the viewbox
    }

    headers = {
        "User-Agent": "local-geocoder/1.0",
    }

    session = get_session()

    try:
        r = session.get(
            base_url,
            params=params,
            headers=headers,
            timeout=(3, 15),  # connect, read
        )

        r.raise_for_status()

        data = r.json()

        if not data:
            return {"latitude": None, "longitude": None, "display_name": None}

        result = data[0]

        return {
            "latitude": result.get("lat"),
            "longitude": result.get("lon"),
            "display_name": result.get("display_name"),
        }

    except Exception as e:
        print(f"Geocode failed: {address} -> {e}")

        return {
            "latitude": None,
            "longitude": None,
            "display_name": None,
        }

In [22]:
address = "123 Summer St, Worcester, MA 01608"
geocode(address)

{'latitude': '42.2649683',
 'longitude': '-71.7965540',
 'display_name': 'Saint Vincent Hospital at Worcester Medical Center, 123, Summer Street, Central Business District, Belmont Hill, Worcester, Worcester County, Massachusetts, 01608, United States'}

In [6]:
address = "96 WATER ST Worcester MA"
geocode(address)

{'latitude': '42.2566275',
 'longitude': '-71.7961191',
 'display_name': '86;96, Water Street, Green Island, Canal District, Worcester, Worcester County, Massachusetts, 01608, United States'}